# Bunch commands for pyTao

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%pylab --no-import-all inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pytao import Tao

# Initialize Tao on the CSR beam tracking example

In [6]:
tao=Tao('-init $ACC_ROOT_DIR/tao/examples/csr_beam_tracking/tao.init -noplot')   

In [ ]:
__x = 1

class A:
    
    __
    
    print(__x)
    
    def __x(self):
        print('here')

## bunch1

In [ ]:
?tao.bunch1

In [ ]:
# The beam is saved at the end. 
stats = tao.bunch1('end')
stats.keys()

In [ ]:
bdata['beam_saved']

In [ ]:
tao.bunch1('end', coordinate='state')

In [ ]:
xdat = tao.cmd_real('python bunch1 1@0>>8|model 1 x')
pxdat = tao.cmd_real('python bunch1 1@0>>8|model 1 px')
ydat = tao.cmd_real('python bunch1 1@0>>8|model 1 y')
pydat = tao.cmd_real('python bunch1 1@0>>8|model 1 py')
zdat = tao.cmd_real('python bunch1 1@0>>8|model 1 z')
pzdat = tao.cmd_real('python bunch1 1@0>>8|model 1 pz')
chargedat = tao.cmd_real('python bunch1 1@0>>8|model 1 charge')
p0cdat = tao.cmd_real('python bunch1 1@0>>8|model 1 p0c')
state = tao.cmd_integer('python bunch1 1@0>>8|model 1 state') # integers
# Unique p0C (eV), charge (pC)
set(p0cdat), sum(chargedat)*1e12

## Plot in matplotlib

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
xdata = 1000*xdat
ydata = 1000*pxdat
weights = chargedat

In [ ]:
# hist2d

mycmap = plt.get_cmap('plasma') # viridis plasma inferno magma and _r versions
mycmap.set_under(color='white') # map 0 to this color
myvmin = 1e-30 # something tiny
# Bin particles
plt.hist2d(x=1000*xdata, y=ydata,  bins=2*[40], weights=weights, cmap=mycmap, vmin=myvmin)
plt.xlabel('x (mm)')
plt.ylabel('px (mrad)')
plt.show()

In [ ]:
# Numpy histogram 2d, with custom color map

import matplotlib.colors as colors

mycmap = plt.get_cmap('viridis') # viridis plasma inferno magma and _r versions
mycmap.set_under(color='white') # map 0 to this color
H, xedges, yedges = np.histogram2d(xdata, ydata, weights=chargedat, bins=40)

xmin, xmax = min(xedges), max(xedges)
ymin, ymax = min(yedges), max(yedges)

image = np.flip(H.T, axis=0) # 
imax = np.max(image)
norm=colors.Normalize(vmin=1e-12*imax, vmax=imax)
plt.xlabel('x (mm)')
plt.ylabel('px (mrad)')
plt.imshow(image, cmap=mycmap, norm=norm, extent=[xmin, xmax, ymin, ymax], aspect='auto')

In [ ]:
np.min(image), np.max(image)

## Plot in Bokeh

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh import palettes, colors
from bokeh.models import ColumnDataSource, HoverTool
output_notebook(verbose=False, hide_banner=True)

pal = palettes.Viridis[256]
#white=colors.named.white
#pal[0] = white # replace 0 with white

In [ ]:
H, xedges, yedges = np.histogram2d(xdata, ydata, weights=chargedat, bins=40)
xmin, xmax = min(xedges), max(xedges)
ymin, ymax = min(yedges), max(yedges)

In [ ]:
ds = ColumnDataSource(data=dict(image=[H.transpose()]))   
p = figure(x_range=[xmin, xmax], y_range=[ymin, ymax], title='Bunch at end', 
           x_axis_label='z (mm)', y_axis_label='pz (1e-3)',
           plot_width=500, plot_height=500)
p.image(image='image', source=ds, 
    x=xmin, y=ymin, dw=xmax-xmin, dh=ymax-ymin, palette=pal)
show(p)

## Data for ParticleGroup

The above commands have been packaged into two functions for easier use, and to easily create ParticleGroup objects

In [ ]:
from pytao.tao_ctypes.bunch import get_bunch_stats, get_bunch_data

In [ ]:
# Get typical stats
stats = get_bunch_stats(tao, 'end')
stats

In [ ]:
# Get data
data = get_bunch_data(tao, 'end')    
data.keys()

In [ ]:
from pmd_beamphysics import ParticleGroup

P = ParticleGroup(data=data)
P

In [ ]:
P.plot('x', 'px')

In [ ]:
# Note the z is 0 by definition
P['mean_z']

In [ ]:
# t it the total time
P['mean_t']*P['mean_beta']*299792458, stats['s']

In [ ]:
P['norm_emit_x'], P['norm_emit_y']

In [ ]:
stats['norm_emit_x'], stats['norm_emit_y']